# DS-GA 1011 Homework 1 - Part 3
## N-Gram Language Modeling

In [2]:
import os
import json
import numpy as np
from collections import defaultdict

#### Utilities

In [3]:
def load_wikitext(filename='wikitext2-sentencized.json'):
    if not os.path.exists(filename):
        !wget "https://nyu.box.com/shared/static/9kb7l7ci30hb6uahhbssjlq0kctr5ii4.json" -O $filename
    
    datasets = json.load(open(filename, 'r'))
    for name in datasets:
        datasets[name] = [x.split() for x in datasets[name]]
    vocab = list(set([t for ts in datasets['train'] for t in ts]))      
    print("Vocab size: %d" % (len(vocab)))
    return datasets, vocab

def perplexity(model, sequences):
    n_total = 0
    logp_total = 0
    for sequence in sequences:
        logp_total += model.sequence_logp(sequence)
        n_total += len(sequence) + 1  
    ppl = 2 ** (- (1.0 / n_total) * logp_total)  
    return ppl

### Additive Smoothing

Fill in the ngram_prob method in the class below:

In [4]:
class NGramAdditive(object):
    def __init__(self, n, delta, vsize):
        self.n = n
        self.delta = delta
        self.count = defaultdict(lambda: defaultdict(float))
        self.total = defaultdict(float)
        self.vsize = vsize
    
    def estimate(self, sequences):
        for sequence in sequences:
            padded_sequence = ['<bos>']*(self.n-1) + sequence + ['<eos>']
            for i in range(len(padded_sequence) - self.n+1):
                ngram = tuple(padded_sequence[i:i+self.n])
                prefix, word = ngram[:-1], ngram[-1]
                self.count[prefix][word] += 1
                self.total[prefix] += 1
                
    def sequence_logp(self, sequence):
        padded_sequence = ['<bos>']*(self.n-1) + sequence + ['<eos>']
        total_logp = 0
        for i in range(len(padded_sequence) - self.n+1):
            ngram = tuple(padded_sequence[i:i+self.n])
            total_logp += np.log2(self.ngram_prob(ngram))
        return total_logp

    def ngram_prob(self, ngram):
        # write me
        prefix = ngram[:-1]
        word = ngram[-1]
        prob = (self.delta + self.count[prefix][word]) / (self.total[prefix] + self.delta * self.vsize)
        return prob

<img src="1.png" width=600 height=600 />

In [5]:
datasets, vocab = load_wikitext()

Vocab size: 33175


In [6]:
delta = 0.0005
for n in [2, 3, 4]:
    lm = NGramAdditive(n=n, delta=delta, vsize=len(vocab)+1)  # +1 is for <eos>
    lm.estimate(datasets['train'])

    print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Train Perplexity: %.3f" % (n, delta, perplexity(lm, datasets['train'])))
    print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Valid Perplexity: %.3f" % (n, delta, perplexity(lm, datasets['valid'])))

Baseline (Additive smoothing, n=2, delta=0.0005)) Train Perplexity: 90.228
Baseline (Additive smoothing, n=2, delta=0.0005)) Valid Perplexity: 525.825
Baseline (Additive smoothing, n=3, delta=0.0005)) Train Perplexity: 26.768
Baseline (Additive smoothing, n=3, delta=0.0005)) Valid Perplexity: 2577.128
Baseline (Additive smoothing, n=4, delta=0.0005)) Train Perplexity: 19.947
Baseline (Additive smoothing, n=4, delta=0.0005)) Valid Perplexity: 9570.901


Is 0.005 a good value for $\delta$? Compare the model's perplexity across several orders of magnitude of $\delta$ (e.g., 0.0005, 0.005, 0.05, 0.5, and 1).

In [7]:
delta = 0.0005
for n in [2, 3, 4]:
    for delta in [0.0005, 0.005, 0.05, 0.5, 1]:
        lm = NGramAdditive(n=n, delta=delta, vsize=len(vocab)+1)  # +1 is for <eos>
        lm.estimate(datasets['train'])

        print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Train Perplexity: %.3f" % (n, delta, perplexity(lm, datasets['train'])))
        print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Valid Perplexity: %.3f" % (n, delta, perplexity(lm, datasets['valid'])))

Baseline (Additive smoothing, n=2, delta=0.0005)) Train Perplexity: 90.228
Baseline (Additive smoothing, n=2, delta=0.0005)) Valid Perplexity: 525.825
Baseline (Additive smoothing, n=2, delta=0.0050)) Train Perplexity: 138.787
Baseline (Additive smoothing, n=2, delta=0.0050)) Valid Perplexity: 475.942
Baseline (Additive smoothing, n=2, delta=0.0500)) Train Perplexity: 331.400
Baseline (Additive smoothing, n=2, delta=0.0500)) Valid Perplexity: 665.990
Baseline (Additive smoothing, n=2, delta=0.5000)) Train Perplexity: 1133.768
Baseline (Additive smoothing, n=2, delta=0.5000)) Valid Perplexity: 1411.721
Baseline (Additive smoothing, n=2, delta=1.0000)) Train Perplexity: 1695.477
Baseline (Additive smoothing, n=2, delta=1.0000)) Valid Perplexity: 1892.272
Baseline (Additive smoothing, n=3, delta=0.0005)) Train Perplexity: 26.768
Baseline (Additive smoothing, n=3, delta=0.0005)) Valid Perplexity: 2577.128
Baseline (Additive smoothing, n=3, delta=0.0050)) Train Perplexity: 116.398
Baseline 

### Interpolation

Implement interpolation smoothing, and find optimal settings of the hyperparameters ($\lambda_1, \ldots, \lambda_n$), for each of $n \in \{2, 3, 4\}$. You can use grid search (i.e., systematically loop through a range of possible values). Show your results, including perplexity for all of the lambda values you have experimented with. Remember: hyperparameter optimization needs to be performed on the validation set -- do not touch the test set until the last section of this notebook!

In [6]:
class NGramInterpolation(object):
    def __init__(self, n, lambdas, vsize, total_word_count):
        self.n = n
        self.lambdas = lambdas
        self.count = defaultdict(lambda: defaultdict(float))
        self.total = defaultdict(float)
        self.vsize = vsize
        self.total_word_count = total_word_count
        
    
    def estimate(self, sequences):
        for sequence in sequences:
            padded_sequence = ['<bos>']*(self.n-1) + sequence + ['<eos>']
            for i in range(self.n-1, len(padded_sequence)):
                ngram = tuple(padded_sequence[i-self.n+1:i+1])
                for j in range(1, self.n):
                    prefix, word = ngram[:j], ngram[j]
                    self.count[prefix][word] += 1
                    self.total[prefix] += 1 
                    self.total[word] += 1
                    
    def sequence_logp(self, sequence):
        padded_sequence = ['<bos>']*(self.n-1) + sequence + ['<eos>']
        total_logp = 0
        for i in range(len(padded_sequence) - self.n+1):
            ngram = tuple(padded_sequence[i:i+self.n])
            total_logp += np.log2(self.ngram_prob(ngram, self.n, self.lambdas[-1]))
        return total_logp

    def ngram_prob(self, ngram, cur_n, cur_lambda):
        prefix, word = ngram[:-1], ngram[-1]
        # we use a recursive way to compute the probability
        # cur_n: the current ngram that we are dealing with
        # cur_lambda: current lambda value
        if cur_n == 1:
            # use additive smoothing when computing the unigram
            prob = (self.total[word] + 0.005) / (self.total_word_count + 0.005 * self.vsize) * cur_lambda
            return prob
        
        if cur_n > 1:
            # self.total[prefix]: the 
            if self.total[prefix] == 0:
                prob = 0
            else:
                prob = self.count[prefix][word] / self.total[prefix]
            return prob * cur_lambda + self.ngram_prob(ngram, cur_n - 1, self.lambdas[cur_n - 2])

<img src="2.png" width=600 height=600 />

In [7]:
total_wc = len([t for ts in datasets['train'] for t in ts])
print(total_wc)

1924754


In [9]:
#Random Search
def grid_search_lambdas(n, n_param_sets):
    lambdas_set = [] 
    for i in range(n_param_sets):
        rand_arr = np.random.rand(1,n)
        normalised_rand_arr = np.round(rand_arr/rand_arr.sum(),2).reshape((n, 1))
        lambdas_set.append(normalised_rand_arr)
    return lambdas_set

In [12]:
for n in [2, 3, 4]:
    best_valid_perplexity = 1e9
    for lambdas in grid_search_lambdas(n, 5):
        lm = NGramInterpolation(n = n, vsize = len(vocab) + 1, total_word_count = total_wc, lambdas = lambdas)  # +1 is for <eos>
        lm.estimate(datasets['train'])
        perplexity_train = perplexity(lm, datasets['train'])
        perplexity_valid = perplexity(lm, datasets['valid'])
        if perplexity_valid < best_valid_perplexity:
            best_valid_perplexity = perplexity_valid
            best_paramset = lambdas
    
        print("Baseline (Interpolation, n=%d, lamdas=%s) Train Perplexity: %.3f" % (n, str(lambdas), perplexity_train))
        print("Baseline (Interpolation, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(lambdas), perplexity_valid))
    print("-"*10)
    print("Best (Interpolation smoothing, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(best_paramset), best_valid_perplexity))
    print()

Baseline (Interpolation, n=2, lamdas=[[0.19]
 [0.81]]) Train Perplexity: 89.162
Baseline (Interpolation, n=2, lamdas=[[0.19]
 [0.81]]) Valid Perplexity: 311.151
Baseline (Interpolation, n=2, lamdas=[[0.12]
 [0.88]]) Train Perplexity: 84.194
Baseline (Interpolation, n=2, lamdas=[[0.12]
 [0.88]]) Valid Perplexity: 330.768
Baseline (Interpolation, n=2, lamdas=[[0.06]
 [0.94]]) Train Perplexity: 80.488
Baseline (Interpolation, n=2, lamdas=[[0.06]
 [0.94]]) Valid Perplexity: 373.063
Baseline (Interpolation, n=2, lamdas=[[0.77]
 [0.23]]) Train Perplexity: 202.676
Baseline (Interpolation, n=2, lamdas=[[0.77]
 [0.23]]) Valid Perplexity: 399.260
Baseline (Interpolation, n=2, lamdas=[[0.5]
 [0.5]]) Train Perplexity: 124.189
Baseline (Interpolation, n=2, lamdas=[[0.5]
 [0.5]]) Valid Perplexity: 315.820
----------
Best (Interpolation smoothing, n=2, lamdas=[[0.19]
 [0.81]]) Valid Perplexity: 311.151

Baseline (Interpolation, n=3, lamdas=[[0.39]
 [0.54]
 [0.07]]) Train Perplexity: 12.191
Baseline (

### Fine tune: n = 2

In [1]:
#Previsous Best Result
# ----------
# Best (Interpolation smoothing, n=2, lamdas=[[0.19]
#  [0.81]]) Valid Perplexity: 311.151

In [ ]:
n=2
lamdas= np.array([[0.19, 0.81]]).reshape((2,1))

step = 0.01 
lambdas_set = []

new_lambdas = lambdas.copy()
for i in range(0, 10): 

    new_lambdas[0][0] = new_lambdas[0][0] - step
    new_lambdas[1][0] = new_lambdas[1][0] + step
    lambdas_set.append(new_lambdas)
    new_lambdas = new_lambdas.copy()
    
new_lambdas = lambdas.copy()
for i in range(0, 10): 

    new_lambdas[0][0] = new_lambdas[0][0] + step
    new_lambdas[1][0] = new_lambdas[1][0] - step
    lambdas_set.append(new_lambdas)
    new_lambdas = new_lambdas.copy()

best_valid_perplexity = 1e9
for lambdas in lambdas_set:
    lm = NGramInterpolation(n = n, vsize = len(vocab) + 1, total_word_count = total_wc, lambdas = lambdas)  # +1 is for <eos>
    lm.estimate(datasets['train'])
    perplexity_train = perplexity(lm, datasets['train'])
    perplexity_valid = perplexity(lm, datasets['valid'])
    if perplexity_valid < best_valid_perplexity:
        best_valid_perplexity = perplexity_valid
        best_paramset = lambdas
    
    print("Baseline (Interpolation, n=%d, lamdas=%s) Train Perplexity: %.3f" % (n, str(lambdas), perplexity_train))
    print("Baseline (Interpolation, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(lambdas), perplexity_valid))
print("-"*10)
print("Best (Interpolation smoothing, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(best_paramset), best_valid_perplexity))
print()

In [ ]:
n=2
lamdas= np.array([[0.26, 0.74]]).reshape((2,1))

step = 0.01 
lambdas_set = []

    
new_lambdas = lambdas.copy()
for i in range(0, 10): 

    new_lambdas[0][0] = new_lambdas[0][0] + step
    new_lambdas[1][0] = new_lambdas[1][0] - step
    lambdas_set.append(new_lambdas)
    new_lambdas = new_lambdas.copy()

best_valid_perplexity = 1e9
for lambdas in lambdas_set:
    lm = NGramInterpolation(n = n, vsize = len(vocab) + 1, total_word_count = total_wc, lambdas = lambdas)  # +1 is for <eos>
    lm.estimate(datasets['train'])
    perplexity_train = perplexity(lm, datasets['train'])
    perplexity_valid = perplexity(lm, datasets['valid'])
    if perplexity_valid < best_valid_perplexity:
        best_valid_perplexity = perplexity_valid
        best_paramset = lambdas
    
    print("Baseline (Interpolation, n=%d, lamdas=%s) Train Perplexity: %.3f" % (n, str(lambdas), perplexity_train))
    print("Baseline (Interpolation, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(lambdas), perplexity_valid))
print("-"*10)
print("Best (Interpolation smoothing, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(best_paramset), best_valid_perplexity))
print()

### Fine tune: n = 3

In [ ]:
#PREVIOUS BEST Result: 
#Best (Interpolation smoothing, n=3, lamdas=[[0.69]
#  [0.02]
#  [0.29]]) Valid Perplexity: 297.408

In [ ]:
n=3
lambdas= np.array([[0.69, 0.02, 0.29]]).reshape((3,1))
step = 0.01

combinations = []
for i in range(3): 
    ls = [0, 1, 2]
    ls.remove(i)
    for j in ls: 
        combinations.append((i, j))
                            

lambdas_set=[]
for comb in combinations: 
    i, j = comb 
    new_lambdas = lambdas.copy()
    new_lambdas[i][0] = lambdas[i][0] + step
    new_lambdas[j][0] = lambdas[j][0] - step
    lambdas_set.append(new_lambdas)                            

best_valid_perplexity = 1e9
for lambdas in lambdas_set:
    lm = NGramInterpolation(n = n, vsize = len(vocab) + 1, total_word_count = total_wc, lambdas = lambdas)  # +1 is for <eos>
    lm.estimate(datasets['train'])
    perplexity_train = perplexity(lm, datasets['train'])
    perplexity_valid = perplexity(lm, datasets['valid'])
    if perplexity_valid < best_valid_perplexity:
        best_valid_perplexity = perplexity_valid
        best_paramset = lambdas
    
    print("Baseline (Interpolation, n=%d, lamdas=%s) Train Perplexity: %.3f" % (n, str(lambdas), perplexity_train))
    print("Baseline (Interpolation, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(lambdas), perplexity_valid))
print("-"*10)
print("Best (Interpolation smoothing, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(best_paramset), best_valid_perplexity))
print()

In [ ]:
#no better off
lambdas= np.array([[0.7, 0.00, 0.3]]).reshape((3,1))

lambdas_set=[lambdas]

best_valid_perplexity = 1e9
for lambdas in lambdas_set:
    lm = NGramInterpolation(n = n, vsize = len(vocab) + 1, total_word_count = total_wc, lambdas = lambdas)  # +1 is for <eos>
    lm.estimate(datasets['train'])
    perplexity_train = perplexity(lm, datasets['train'])
    perplexity_valid = perplexity(lm, datasets['valid'])
    if perplexity_valid < best_valid_perplexity:
        best_valid_perplexity = perplexity_valid
        best_paramset = lambdas
    
    print("Baseline (Interpolation, n=%d, lamdas=%s) Train Perplexity: %.3f" % (n, str(lambdas), perplexity_train))
    print("Baseline (Interpolation, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(lambdas), perplexity_valid))
print("-"*10)
print("Best (Interpolation smoothing, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(best_paramset), best_valid_perplexity))
print()

### Fine tune: n = 4

In [ ]:
#PREVIOUS BEST Result:
# Best (Interpolation smoothing, n=4, lamdas=[[0.46]
#  [0.18]
#  [0.33]
#  [0.03]]) Valid Perplexity: 410.779

In [ ]:
n=4
lambdas= np.array([[0.46, 0.18, 0.33, 0.03]]).reshape((4,1))
step = 0.01

combinations = []
for i in range(4): 
    ls = [0, 1, 2, 3]
    ls.remove(i)
    for j in ls: 
        combinations.append((i, j))
                            

lambdas_set=[]
for comb in combinations: 
    i, j = comb 
    new_lambdas = lambdas.copy()
    new_lambdas[i][0] = lambdas[i][0] + step
    new_lambdas[j][0] = lambdas[j][0] - step
    lambdas_set.append(new_lambdas)                            


In [ ]:
best_valid_perplexity = 1e9
for lambdas in lambdas_set:
    lm = NGramInterpolation(n = n, vsize = len(vocab) + 1, total_word_count = total_wc, lambdas = lambdas)  # +1 is for <eos>
    lm.estimate(datasets['train'])
    perplexity_train = perplexity(lm, datasets['train'])
    perplexity_valid = perplexity(lm, datasets['valid'])
    if perplexity_valid < best_valid_perplexity:
        best_valid_perplexity = perplexity_valid
        best_paramset = lambdas
    
    print("Baseline (Interpolation, n=%d, lamdas=%s) Train Perplexity: %.3f" % (n, str(lambdas), perplexity_train))
    print("Baseline (Interpolation, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(lambdas), perplexity_valid))
print("-"*10)
print("Best (Interpolation smoothing, n=%d, lamdas=%s) Valid Perplexity: %.3f" % (n, str(best_paramset), best_valid_perplexity))
print()

### Optimal Parameter Sets

In [ ]:
# Optimal Parameter Sets After Fine Tune

# n = 4
# ----------
# Best (Interpolation smoothing, n=4, lamdas=[[0.47]
#  [0.17]
#  [0.33]
#  [0.03]]) Valid Perplexity: 404.623
# 3
# n = 3
# ----------
# Best (Interpolation smoothing, n=3, lamdas=[[0.7 ]
#  [0.01]
#  [0.29]]) Valid Perplexity: 297.266
# n = 2
# ----------
# Best (Interpolation smoothing, n=2, lamdas=[[0.32]
#  [0.68]]) Valid Perplexity: 301.697

Now compare the performance of the two smoothing methods on the test set. Use the best hyperparameters you found for each.

In [15]:
#The best additive smoothing model would be n = 2, delta = 0.005 
lm_add = NGramAdditive(n = 2, delta = 0.005, vsize = len(vocab) + 1)  # +1 is for <eos>
lm_add.estimate(datasets['train'])

print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Valid Perplexity: %.3f" % (2, 0.005, perplexity(lm_add, datasets['valid'])))
print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Test Perplexity: %.3f" % (2, 0.005, perplexity(lm_add, datasets['test'])))

Baseline (Additive smoothing, n=2, delta=0.0050)) Valid Perplexity: 475.942
Baseline (Additive smoothing, n=2, delta=0.0050)) Test Perplexity: 447.054


In [10]:
#The best interpolation model in our grid search would be n = 3, lambda = [0.69, 0.02, 0.29]
lambdas = [0.7, 0.01, 0.29]
lm_int = NGramInterpolation(n = 3, vsize = len(vocab) + 1, total_word_count = total_wc, lambdas = lambdas)  # +1 is for <eos>
lm_int.estimate(datasets['train'])

print("Baseline (Interpolation, n=%d, lamdas=%s)) Valid Perplexity: %.3f" % (2, str(lambdas), perplexity(lm_int, datasets['valid'])))
print("Baseline (Interpolation, n=%d, lamdas=%s)) Test Perplexity: %.3f" % (2, str(lambdas), perplexity(lm_int, datasets['test'])))

Baseline (Interpolation, n=2, lamdas=[0.7, 0.01, 0.29])) Valid Perplexity: 297.266
Baseline (Interpolation, n=2, lamdas=[0.7, 0.01, 0.29])) Test Perplexity: 282.715


**Answer**:    
From the above result we can see that the interpolation model performs better than the additive model. It is because interpolation model is a combination of different ngrams and it also leverage the trick of additive smoothing when the count of word in unigrm is zero.